## Adding atoms to read file
We will hydrogenate the MoS2 monolayer with random percentage in this exercise

In [1]:
from ase.io import lammpsdata
import ase.build as build
import numpy as np
lmp_data = lammpsdata.read_lammps_data("./data/2H-MoS2-in.lmp",
                                       style="charge",
                                       units="real")
atomic_positions = lmp_data.get_positions()
atomic_numbers = lmp_data.get_atomic_numbers()

sorted_lmp_data = build.sort(lmp_data, tags=-atomic_positions[:, 2])
lammpsdata.write_lammps_data("./data/2H-MoS2-sorted-out.lmp",
                             sorted_lmp_data,
                             atom_style="charge")

In [2]:
H_percent = 80
H_S_bl = 1.4  # Hydrogen-Sulfer bond length

# We first extract the position of sulpher atoms
# Sulfer atoms are type 2 and so the atomic no of sulfer is 2 in
# the read file
S_positions = atomic_positions[np.where(atomic_numbers == 2)[0]]

# We calculate the number of H-atoms to be added based on the H-percentage
S_no = S_positions.shape[0]  # Total number of Sulfer atoms
H_no = S_no*H_percent//100  # We obtain the integer number

S_no, H_no, S_positions

(5520,
 4416,
 array([[  1.08293784,   1.64164494,   8.40662106],
        [  1.06852429,   4.83062714,   8.41133573],
        [  1.06547351,   8.01899   ,   8.42039964],
        ...,
        [246.66646294,  92.44248085,  11.5760478 ],
        [252.17490914,  95.63388543,   8.40959402],
        [241.12903734,  79.71525134,  11.57009069]]))

We have Sulfer in two layers. top(11.5) bottom(8.4)
We need to add sulfer above the top layer and below the bottom layer.
#### We now have two possibility of random H-addition.
1. No of H-atoms on top and bottom layer are also random
2. Approx equal no of H-atom on top and bottom layer

We start with the second case

In [3]:
H_no_top = H_no//2
H_no_bot = H_no - H_no_top
H_no_top, H_no_bot

S_positions_top = S_positions[S_positions[:, 2] > 10]
S_positions_bot = S_positions[S_positions[:, 2] < 10]

np.random.seed(1)  # To reproduce the results
top_indicies = np.random.choice(S_positions_top.shape[0],
                                H_no_top,
                                replace=False)
bot_indicies = np.random.choice(S_positions_bot.shape[0],
                                H_no_bot,
                                replace=False)
top_indicies.shape, bot_indicies.shape

((2208,), (2208,))

Now we are ready to add Hydrogen. ASE can not write atomic masses to lammpsdata file. We will add Li instead to H because it have atomic number 3
We already have two atom types already in the list

In [4]:
for i in top_indicies:
    # it is better to add Li as here becuase we know
    # H will be the third element except Mo and S
    lmp_data.append("Li")
    lmp_data.positions[-1] = S_positions_top[i]+np.array([0, 0, H_S_bl])
for i in bot_indicies:
    lmp_data.append("Li")
    lmp_data.positions[-1] = S_positions_bot[i]+np.array([0, 0, -H_S_bl])
lmp_data.set_initial_charges(lmp_data.get_initial_charges()*0)

In [5]:
lammpsdata.write_lammps_data(f"./data/2H-MoS2-H_{H_percent}-out.lmp",
                             lmp_data,
                             atom_style="charge")